### Generating Features - Writing Analysis

Dependencies

In [ ]:
import os, csv, pandas as pd, string
from nltk.metrics import edit_distance as ed

In [ ]:
word_dict = {
    'the': 1,
    'beautiful': 2,
    'ducks': 3,
    'swam': 4,
    'in': 5,
    'big': 6,
    'blue': 7,
    'lake': 8,
    'their': 9,
    'feathers': 10,
    'were': 11,
    'very': 12,
    'pretty': 13,
    'and': 14,
    'quite': 15,
    'often': 16,
    'would': 17,
    'jump': 18,
    'out': 19,
    'of': 20,
    'water': 21,
}
word_dict_key = list(word_dict.keys())
sim_con = {
    'b': ['d','h','q','p'],
    'd': ['b', 'h', 'q','p'],
    'h': ['b', 'd', 'q', 'p'],
    'q': ['b', 'd', 'h', 'p'],
    'p': ['b', 'd', 'h', 'q'],
    'f': ['t'],
    't': ['f'],
}
sim_vow = {
    'o': ['u', 'e'],
    'i': ['e'],
    'a': ['e'],
    'e': ['i', 'a', 'u']
}
consonants = ['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']
vowels = ['a', 'e', 'i', 'o', 'u']
# print(word_dict_key)

Spelling Features

In [ ]:
def sim_characters(src, tgt):
    score = []
    phe = 0
    unv = 0
    aee = 0
    doc = 0

    # if length of src and tgt is same but the error is in character
    if len(src) == len(tgt):
        for c in range(len(src)):

            if src[c] != tgt[c]:
                # phonetic error [6]
                if src[c] in sim_con.keys() and tgt[c] in sim_con[src[c]]: phe += 1

                #unstressed vowel [10]
                elif src[c] in sim_vow.keys() and tgt[c] in sim_vow[src[c]]: unv += 1

    # if length of tgt is greater, meaning the child has written extra character
    elif len(src) < len(tgt):
        
        prev_c = ''
        for c in range(len(tgt)):

            # adding extra e [1]
            if c == len(tgt) - 1:
                if tgt[-1] == 'e' and src[-1] != 'e': aee += 1

            if c < len(src):
                #phonetic error [6]
                if src[c] in sim_con.keys() and tgt[c] in sim_con[src[c]]: phe += 1
                
                #doubling of consonants [3]
                if src[c] != tgt[c] and tgt[c] == prev_c and tgt[c] in consonants: doc += 1
                elif src[c] == tgt[c]: prev_c = src[c]

    return phe, unv, aee, doc

def scoring(df):

    sentence_length = list()
    PHE = []    #phonetic error [6]
    UNV = []    #unstressed vowel [10]
    AEE = []    #added extra e [1]
    DOC = []    #doubling of consonants [3]

    for ind, row in df.iterrows():
        phe, unv, aee, doc = 0, 0, 0, 0
        words = row['sentence'].translate(str.maketrans('','',string.punctuation)).lower()
        words = words.split()
        
        #length of sentence
        if len(words) == 25: sentence_length.append(0)
        else: sentence_length.append(abs(len(words)-25))

        for word in words:
            src = ''
            tgt = word
            min_dist = 999

            # edit distance to find the closest word if there are spelling errors
            for x in word_dict_key:
                dist = ed(word, x)
                if dist < min_dist:
                    min_dist = dist
                    src = x
            if min_dist > 0:   # there is an error 
                a, b, c, d = sim_characters(src, tgt)
                phe += a
                unv += b
                aee += c
                doc += d
            
        PHE.append(phe)
        UNV.append(unv)
        AEE.append(aee)
        DOC.append(doc)

    df['sentence length'] = sentence_length
    df['phonetic error'] = PHE
    df['unstressed vowel'] = UNV
    df['adding extra e'] = AEE
    df['doubling consonant'] = DOC
    
    return df

Vocabulary Features

In [ ]:
import re

def analyse_text(sentences):
    vocab_lst = []
    complexity_lst = []
    for s in sentences:
        # Compute the vocabulary length and complexity
        words = re.findall(r'\b\w+\b', s.lower())
        unique_words = set(words)
        vocab_length = len(unique_words)
        complexity = sum(1 for word in unique_words if words.count(word) > 1)
        vocab_lst.append(vocab_length)
        complexity_lst.append(complexity)
    return vocab_lst, complexity_lst

Read data, generate features, and save as a CSV

In [ ]:
df = pd.read_csv('data.csv')

df = scoring(df)
sentences = list(df['sentence'])
v, c = analyse_text(sentences)
df['vocab_length'] = v
df['complexity'] = c

df.to_csv('data.csv', index=False)
print(df)

### Training Binary Classifiers

Dependencies

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Read data and Prepare for Model Training

In [ ]:
# Read the CSV file
data = pd.read_csv('data.csv')

# Drop the 'id' column
data = data.drop('id', axis=1)
data = data.drop('sentence', axis=1)
data = data.drop('label', axis=1)


# Split the data into features (X) and target (y)
X = data.drop('class', axis=1)
y = data['class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

SVM and Logistic Regression

In [ ]:
# Train an SVM classifier
svm_classifier = SVC()
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set using the SVM classifier
svm_predictions = svm_classifier.predict(X_test)

# Calculate evaluation metrics for the SVM classifier
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_precision = precision_score(y_test, svm_predictions)
svm_recall = recall_score(y_test, svm_predictions)
svm_f1 = f1_score(y_test, svm_predictions)

# Train a logistic regression classifier
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train, y_train)

# Make predictions on the test set using the logistic regression classifier
logreg_predictions = logreg_classifier.predict(X_test)

# Calculate evaluation metrics for the logistic regression classifier
logreg_accuracy = accuracy_score(y_test, logreg_predictions)
logreg_precision = precision_score(y_test, logreg_predictions)
logreg_recall = recall_score(y_test, logreg_predictions)
logreg_f1 = f1_score(y_test, logreg_predictions)

# Print the evaluation metrics
print("SVM Classifier:")
print("Accuracy:", svm_accuracy)
print("Precision:", svm_precision)
print("Recall:", svm_recall)
print("F1 Score:", svm_f1)
print()
print("Logistic Regression Classifier:")
print("Accuracy:", logreg_accuracy)
print("Precision:", logreg_precision)
print("Recall:", logreg_recall)
print("F1 Score:", logreg_f1)


Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
# Train a Naive Bayes classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

# Make predictions on the test set using the Naive Bayes classifier
nb_predictions = nb_classifier.predict(X_test)

# Calculate evaluation metrics for the Naive Bayes classifier
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_precision = precision_score(y_test, nb_predictions)
nb_recall = recall_score(y_test, nb_predictions)
nb_f1 = f1_score(y_test, nb_predictions)

# Generate the correlation matrix
correlation_matrix = data.drop('class', axis=1).corr()

# Print the evaluation metrics
print("Naive Bayes Classifier:")
print("Accuracy:", nb_accuracy)
print("Precision:", nb_precision)
print("Recall:", nb_recall)
print("F1 Score:", nb_f1)
print()
print("Correlation Matrix:")
print(correlation_matrix)

Random Forest and Decision Tree

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set using the Random Forest classifier
rf_predictions = rf_classifier.predict(X_test)

# Calculate evaluation metrics for the Random Forest classifier
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_precision = precision_score(y_test, rf_predictions)
rf_recall = recall_score(y_test, rf_predictions)
rf_f1 = f1_score(y_test, rf_predictions)

# Train a Decision Tree classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

# Make predictions on the test set using the Decision Tree classifier
dt_predictions = dt_classifier.predict(X_test)

# Calculate evaluation metrics for the Decision Tree classifier
dt_accuracy = accuracy_score(y_test, dt_predictions)
dt_precision = precision_score(y_test, dt_predictions)
dt_recall = recall_score(y_test, dt_predictions)
dt_f1 = f1_score(y_test, dt_predictions)

# Generate the correlation matrix
correlation_matrix = data.drop('class', axis=1).corr()

# Print the evaluation metrics
print("Random Forest Classifier:")
print("Accuracy:", rf_accuracy)
print("Precision:", rf_precision)
print("Recall:", rf_recall)
print("F1 Score:", rf_f1)
print()
print("Decision Tree Classifier:")
print("Accuracy:", dt_accuracy)
print("Precision:", dt_precision)
print("Recall:", dt_recall)
print("F1 Score:", dt_f1)
print()
print("Correlation Matrix:")
print(correlation_matrix)

# Generate the confusion matrix for the Random Forest classifier
rf_confusion_matrix = confusion_matrix(y_test, rf_predictions)

# Generate the confusion matrix for the Decision Tree classifier
dt_confusion_matrix = confusion_matrix(y_test, dt_predictions)

# Print the confusion matrices
print("Random Forest Confusion Matrix:")
print(rf_confusion_matrix)
print()
print("Decision Tree Confusion Matrix:")
print(dt_confusion_matrix)